In [6]:
import pandas as pd
import numpy as np
import trueskillthroughtime as tst
import altair as alt
from scipy.optimize import minimize

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Read in the Warcraft 3 Game Data

Each row gives you a match, which is made up over multiple games. So a match might be the best two out of three. The outcome column indicates who won the overall match. If outcome = 1, competitor_1 won.

In [7]:
games = pd.read_csv('data\warcraft3.csv')
games['date'] = pd.to_datetime(games['date'])

games = games.query('(competitor_1_score > -0.0001) & (competitor_2_score > -0.0001)') # Clean up non sense.

for cs in ['competitor_1_score', 'competitor_2_score']:
    games[cs] = games[cs].astype(int)

# Time, normalized to days and relative to the first game.
games['time_in_days'] = (games['date'] - games['date'].min()).dt.days

# Top players worth inspecting. Happy is the best
top_players_elo = ['Happy', 'Lyn', 'Fortitude', 'Eer0', 'Sok', 'ColorFul', 'Kaho', 'FoCuS', 'Moon',  'LabyRinth', 'Life', 'LawLiet', 'Blade', 'Fly100%', 'Starbuck', 'Soin', 'Infi', 'Sini',  'Dise', 'Chaemiko', 'EleGaNt', 'Leon', 'Lin_Guagua', 'WFZ', 'XiaoKK', 'XlorD', 'ReMinD', 'PaTo', 'HawK']

In [8]:
games.head()

,date,competitor_1,competitor_2,competitor_1_score,competitor_2_score,outcome,match_id,page,time_in_days
0,2002-05-24 00:00:00,Onlyno1,Hide1191,0,2,0.0,pGynPIUygQ_R01-M001,https://liquipedia.net/warcraft/Warcraft_3_Bet...,0
1,2002-05-24 00:00:01,Medusa,Landsoul,2,0,1.0,pGynPIUygQ_R01-M002,https://liquipedia.net/warcraft/Warcraft_3_Bet...,0
2,2002-05-24 00:00:02,BabyHero,Rori,2,0,1.0,pGynPIUygQ_R01-M003,https://liquipedia.net/warcraft/Warcraft_3_Bet...,0
3,2002-05-24 00:00:03,Skelton,JoJo,2,0,1.0,pGynPIUygQ_R01-M004,https://liquipedia.net/warcraft/Warcraft_3_Bet...,0
4,2002-05-24 00:00:04,Hide1191,Medusa,2,1,1.0,pGynPIUygQ_R02-M001,https://liquipedia.net/warcraft/Warcraft_3_Bet...,0


# Shrink the data, so the algorithm runs faster

In [9]:
n_recent_matches = 42500 # Reduce the data by only looking at recent matches.
top_k = 500 # Reduce the data by only looking at the top top_k players.

In [10]:
def get_competitors(games):
    # Create a unique list of all players.
    return list(set(games['competitor_1'].values) | set(games['competitor_2'].values))

def get_top_k_most_playing_players(games, k):
    # Tells you which players played the most.
    competitors = get_competitors(games)
    games_played = (games['competitor_1'].value_counts().reindex(competitors).fillna(0) + games['competitor_2'].value_counts().reindex(competitors).fillna(0)).sort_values(ascending=False).astype(int)
    return games_played, games_played.index[:k].values

games_played, top_k_players = get_top_k_most_playing_players(games, top_k)

games = games.query("(competitor_1 in @top_k_players) & (competitor_2 in @top_k_players)")
games = games.iloc[-n_recent_matches:]

# Print some facts about the data

In [11]:
n_matches = games.shape[0]

competitors = get_competitors(games)
n_competitors = len(competitors)

assert games['outcome'].isnull().mean() == 0

print(f"Number of matches: {n_matches:,}")
print(f"Number of competitors: {n_competitors:,}")
print(f"Min date: {games['date'].min()}")
print(f"Max date: {games['date'].max()}")
print(f"Top ten players: {', '.join(top_k_players[:10])}")
print(f"outcome frequencies: {games['outcome'].value_counts().to_dict()}")
print(f"competitor_1_score frequencies: {games['competitor_1_score'].value_counts().to_dict()}")
print(f"competitor_2_score frequencies: {games['competitor_2_score'].value_counts().to_dict()}")
games.head()

Number of matches: 42,500
Number of competitors: 408
Min date: 2018-06-24 15:00:00
Max date: 2024-09-06 17:00:09
Top ten players: FoCuS, Sonik, HawK, 국사무쌍, Happy, Sheik, Foggy, MisterWinner, LawLiet, Lyn
outcome frequencies: {1.0: 23988, 0.0: 18230, 0.5: 282}
competitor_1_score frequencies: {1: 14845, 0: 14478, 2: 11205, 3: 1607, 4: 273, 5: 47, 6: 26, 7: 16, 8: 2, 9: 1}
competitor_2_score frequencies: {0: 19471, 1: 13210, 2: 8382, 3: 1188, 4: 193, 5: 35, 6: 9, 7: 9, 9: 2, 8: 1}


,date,competitor_1,competitor_2,competitor_1_score,competitor_2_score,outcome,match_id,page,time_in_days
59864,2018-06-24 15:00:00,MysT,Muffy,0,1,0.0,kddfCy7LTI_0002_4,https://liquipedia.net/warcraft/New_WarCraft_3...,5875
59865,2018-06-24 15:00:00,MysT,Muffy,0,1,0.0,kddfCy7LTI_0002_5,https://liquipedia.net/warcraft/New_WarCraft_3...,5875
59878,2018-06-24 15:00:00,Ag3nt,Sheik,1,0,1.0,kddfCy7LTI_0004_3,https://liquipedia.net/warcraft/New_WarCraft_3...,5875
59879,2018-06-24 15:00:00,Ag3nt,Sheik,1,0,1.0,kddfCy7LTI_0004_4,https://liquipedia.net/warcraft/New_WarCraft_3...,5875
59880,2018-06-24 15:00:00,Nick,TGW,1,0,1.0,kddfCy7LTI_0004_5,https://liquipedia.net/warcraft/New_WarCraft_3...,5875


# Put the data into a format the algorithm can accept

This is the `game_position` variable.

Also, create the `wins_losses` variable, just to inspect.

In [12]:
games['time_0_to_999_int'] = ((games['time_in_days'] - games['time_in_days'].min())/(games['time_in_days'].max() - games['time_in_days'].min())*999).astype(int)
competitor_name_map = {c: chr(97+i) for i, c in enumerate(competitors)}
competitor_name_map_inv = {l: c for c, l in competitor_name_map.items()}

In [13]:
wins_losses = pd.DataFrame(0, index=competitors, columns=['wins', 'losses'])
game_composition = []
times = []
np.random.seed(0)

for _, row in games.iterrows():
    c1, c2, c1s, c2s, t = row['competitor_1'], row['competitor_2'], row['competitor_1_score'], row['competitor_2_score'], row['time_0_to_999_int']
    
    wins_losses.loc[c1, 'wins'] += c1s
    wins_losses.loc[c2, 'losses'] += c1s
    wins_losses.loc[c2, 'wins'] += c2s
    wins_losses.loc[c1, 'losses'] += c2s

    c1m, c2m = competitor_name_map[c1], competitor_name_map[c2]
    comp = [(c1m, c2m)] * c1s + [(c2m, c1m)] * c2s
    comp = np.random.permutation(comp).tolist() # Game order matters and we don't actually know it, so we randomize over it.

    for cp_g in comp:
        game_composition.append(cp_g)
        times.append(t)

wins_losses = wins_losses.assign(games = wins_losses.sum(axis=1), win_percent = wins_losses['wins'] / wins_losses.sum(axis=1)).sort_values(by='win_percent', ascending=False)
wins_losses.head(8)

,wins,losses,games,win_percent
Believe,1,0,1,1.000000
Space,3,0,3,1.000000
Happy,3919,868,4787,0.818676
Padash,7,2,9,0.777778
Sweet,3,1,4,0.750000
Hitman,1088,394,1482,0.734143
XlorD,533,221,754,0.706897
DNA,7,3,10,0.700000


## Optimize the hyperparameters of the algorithm

Because TrueSkill is a Bayesian algorithm, we can calculate marginal likelihood (the log evidence) over the parameters, as a function of the hyperparameters (gamma, sigma, beta). The below is the optimization to do this. It takes awhile to run, so it is commented out. 

We are using this package: https://github.com/glandfried/TrueSkillThroughTime.py

In [14]:
# def negative_log_evidence(params):
#     gamma, sigma, beta = params
#     print(f"gamma: {gamma:.4f}, sigma: {gamma:.4f}, beta: {gamma:.4f}, ")
#     history = tst.History(game_composition, times=times, gamma=gamma, sigma=sigma, beta=beta)
#     nle = -history.log_evidence()
#     print(f"NLE: {nle:.4f}")
#     return nle

# # Initial guesses for gamma and sigma
# initial_params = [.02, 5.0, 1]

# # Define the bounds for both gamma and sigma (in this case, between 0 and 10 for each)
# bounds = [(0.00001, 0.1), (0.0001, 8), (0.0001, 8)]

# # Perform the optimization with bounds
# result = minimize(negative_log_evidence, initial_params, bounds=bounds)

# # Print the result
# optimal_gamma, optimal_sigma, optimal_beta = result.x
# print(f"optimal_gamma =  {optimal_gamma}")
# print(f"optimal_sigma = {optimal_sigma}")
# print(f"optimal_beta = {optimal_beta}")

## Harcoded optimal hyperparameters

In [15]:
optimal_gamma =  0.08105500545076469
optimal_sigma = 3.738460608607473
optimal_beta = 2.7614029558670308

## Run the TrueSkillThroughTime algorithm

In [16]:
history = tst.History(game_composition, times=times, gamma=optimal_gamma, sigma=optimal_sigma, beta=optimal_beta)
learning_curves = history.learning_curves() #
assert set(learning_curves.keys()) == set(competitor_name_map_inv.keys())
-history.log_evidence()

46329.8565709718

## Plot TrueSkills over time

In [17]:
top_players_elo_pick = top_players_elo[:20]
curves = {p: learning_curves[competitor_name_map[p]][20:] for p in competitors}

In [18]:
t_ints_to_date = games.groupby('time_0_to_999_int')['date'].mean().dt.date.to_dict()

In [19]:
# Create a dataframe for each player and combine them
df_list = []
for player, data in curves.items():
    if player not in top_players_elo_pick:
        continue
    df = pd.DataFrame({
        'date': [t_ints_to_date[t] for t, _ in data],
        'mu': [d.mu for _, d in data],
        'sigma': [d.sigma for _, d in data]
    })
    df['player'] = player
    df['lower'] = df['mu'] - 1 * df['sigma']
    df['upper'] = df['mu'] + 1 * df['sigma']
    if player == 'Lyn':
        df_ch = df
    df_list.append(df)

df_combined = pd.concat(df_list)
df_combined["date"] = pd.to_datetime(df_combined["date"])

line = alt.Chart(df_combined).mark_line(strokeWidth=4).encode(
    x=alt.X('date:T', axis=alt.Axis(format="%Y %B")),
    y=alt.Y('mu', scale=alt.Scale(zero=False), title='Skill'),
    color=alt.Color('player', sort=top_players_elo)
)
confidence_interval = alt.Chart(df_combined).mark_area(opacity=0.4).encode(
    x='date',
    y='lower',
    y2='upper',
    color='player'
)

chart = confidence_interval + line
chart.properties(width=1000, height=800, title='Skills over Time with 1 Std bands').configure_axis(grid=False)

alt.LayerChart(...)

## Create Calibration Plots

We answer the question: If TrueSkill thinks the probability of player 1 winning is 30%, how often does player 1 actually win? If it's a well calibrated model, it should around 30%. That's what this checks.

In [20]:
import numpy as np
from scipy.integrate import quad

# Define the normal distribution PDF
def normal_pdf(x, mu, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

Here, we create the model probabilities of wins.

In [21]:
curves_map = {k: {t: n for t, n in v} for k, v in curves.items()}
games['win_prob'] = np.nan

for i, row in games.iterrows():
    c1, c2, t_int = row['competitor_1'], row['competitor_2'], row['time_0_to_999_int']
    if c1 in curves_map and c2 in curves_map:
        if t_int in curves_map[c1] and t_int in curves_map[c2]:
            normal_1, normal_2 = curves_map[c1][t_int], curves_map[c2][t_int]
            mu_diff = normal_1.mu - normal_2.mu
            sigma2_diff = normal_1.sigma**2 + normal_2.sigma**2 + 2*(optimal_beta**2)
            result, error = quad(normal_pdf, 0, np.inf, args=(mu_diff, sigma2_diff**.5))
            games.loc[i, 'win_prob'] = result

In [22]:
print(games[games['win_prob'].notnull()].head().to_string())

                     date  competitor_1 competitor_2  competitor_1_score  competitor_2_score  outcome               match_id                                              page  time_in_days  time_0_to_999_int  win_prob
61221 2018-09-27 03:00:00      ColorFul      LawLiet                   0                   1      0.0  hhpCcxc5LX_R01-M001_7  https://liquipedia.net/warcraft/NeXT/2018_Summer          5970                 41  0.364109
61222 2018-09-27 03:00:00      ColorFul      LawLiet                   1                   0      1.0  hhpCcxc5LX_R01-M001_8  https://liquipedia.net/warcraft/NeXT/2018_Summer          5970                 41  0.364109
61223 2018-09-27 03:00:00      ColorFul      LawLiet                   0                   1      0.0  hhpCcxc5LX_R01-M001_9  https://liquipedia.net/warcraft/NeXT/2018_Summer          5970                 41  0.364109
61230 2018-09-28 04:00:00      ColorFul      LawLiet                   1                   0      1.0  bAgNh8QMRs_R01-M001_6  ht

In [23]:
games['n_games'] = games['competitor_1_score'] + games['competitor_2_score']
games['win_percent'] = games['competitor_1_score'] / games['n_games']

In [24]:
df = games[games['win_prob'].notnull()]

df['win_prob_bucket'] = pd.qcut(df['win_prob'], q=5, duplicates='drop')

bucket_means = df.groupby('win_prob_bucket').agg(
    avg_outcome=('win_percent', 'mean'),
    win_prob_midpoint=('win_prob', 'mean')
)

calibration_plot = alt.Chart(bucket_means).mark_line().encode(
    x=alt.X('win_prob_midpoint', title='Win Probability'),
    y=alt.Y('avg_outcome', title='Average Outcome'),
    # tooltip=['win_prob_midpoint', 'avg_outcome']
).properties(
    title='Calibration Plot'
)
calibration_plot

C:\Users\Duane\AppData\Local\Temp\ipykernel_41988\2406384203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['win_prob_bucket'] = pd.qcut(df['win_prob'], q=5, duplicates='drop')


alt.Chart(...)